<h3>Importing Libraries</h3>

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<h3>Scraping data to a DataFrame using Pandas</h3>

In [2]:
# Dataset link: https://geo.nyu.edu/catalog/nyu_2451_34572

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

<h3>Getting the geographical coordinates</h3>

In [3]:
neighborhoods_data = newyork_data['features']

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


<h3>Creating a map of New York with neighborhoods superimposed on top</h3>

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688

In [11]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<h3>Define Foursquare Credentials and Version</h3>

In [15]:
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EARGWUC1PTDGHJFJDAEXRPU53IPNXVDI3NILWKBTADHVTRAA
CLIENT_SECRET:YDB2ZBCSVM2GSCNQ0TU53NN3QF3EMKMTZHS4WBAEFPQXQMIM


<h3>Getting the top 100 venues that are within a radius of 500 meters.</h3>

In [16]:
radius = 500
LIMIT = 100

venues = []

for lat, lon, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:
manhattan_venues_df = pd.DataFrame(venues)

manhattan_venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

print(manhattan_venues_df.shape)
manhattan_venues_df.head()

(3201, 7)


Neighborhood   Latitude  Longitude          Venue  Venue Latitude  \
0  Marble Hill  40.876551  -73.91066       Arturo's       40.874412   
1  Marble Hill  40.876551  -73.91066    Bikram Yoga       40.876844   
2  Marble Hill  40.876551  -73.91066  Tibbett Diner       40.880404   
3  Marble Hill  40.876551  -73.91066        Dunkin'       40.877136   
4  Marble Hill  40.876551  -73.91066      Starbucks       40.877531   

   Venue Longitude Venue Category  
0       -73.910271    Pizza Place  
1       -73.906204    Yoga Studio  
2       -73.908937          Diner  
3       -73.906666     Donut Shop  
4       -73.905582    Coffee Shop

In [18]:
manhattan_venues_df.groupby(["Neighborhood"]).count()

Latitude  Longitude  Venue  Venue Latitude  \
Neighborhood                                                      
Battery Park City          65         65     65              65   
Carnegie Hill              86         86     86              86   
Central Harlem             45         45     45              45   
Chelsea                   100        100    100             100   
Chinatown                 100        100    100             100   
Civic Center              100        100    100             100   
Clinton                   100        100    100             100   
East Harlem                40         40     40              40   
East Village              100        100    100             100   
Financial District        100        100    100             100   
Flatiron                  100        100    100             100   
Gramercy                   94         94     94              94   
Greenwich Village         100        100    100             100   
Hamilton Heights           63         63     63              63   
Hudson Yards               62         62     62              62   
Inwood                     56         56     56              56   
Lenox Hill                100        100    100             100   
Lincoln Square             92         92     92              92   
Little Italy              100        100    100             100   
Lower East Side            47         47     47              47   
Manhattan Valley           49         49     49              49   
Manhattanville             46         46     46              46   
Marble Hill                22         22     22              22   
Midtown                   100        100    100             100   
Midtown South             100        100    100             100   
Morningside Heights        43         43     43              43   
Murray Hill               100        100    100             100   
Noho                      100        100    100             100   
Roosevelt Island           26         26     26              26   
Soho                      100        100    100             100   
Stuyvesant Town            17         17     17              17   
Sutton Place              100        100    100             100   
Tribeca                    88         88     88              88   
Tudor City                 81         81     81              81   
Turtle Bay                100        100    100             100   
Upper East Side            97         97     97              97   
Upper West Side            98         98     98              98   
Washington Heights         84         84     84              84   
West Village              100        100    100             100   
Yorkville                 100        100    100             100   

                     Venue Longitude  Venue Category  
Neighborhood                                          
Battery Park City                 65              65  
Carnegie Hill                     86              86  
Central Harlem                    45              45  
Chelsea                          100             100  
Chinatown                        100             100  
Civic Center                     100             100  
Clinton                          100             100  
East Harlem                       40              40  
East Village                     100             100  
Financial District               100             100  
Flatiron                         100             100  
Gramercy                          94              94  
Greenwich Village                100             100  
Hamilton Heights                  63              63  
Hudson Yards                      62              62  
Inwood                            56              56  
Lenox Hill                       100             100  
Lincoln Square                    92              92  
Little Italy                     100             100  
Lower East Side                   47              4

In [19]:
manhattan_venues_df['Venue Category'].unique()[:100]

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Donut Shop', 'Coffee Shop',
       'Gym', 'Department Store', 'Tennis Stadium', 'Pharmacy',
       'Seafood Restaurant', 'Supplement Shop', 'Ice Cream Shop',
       'Sandwich Place', 'Discount Store', 'Video Game Store',
       'Steakhouse', 'Kids Store', 'Deli / Bodega', 'Hotel',
       'Cocktail Bar', 'Greek Restaurant', 'Bakery', 'Chinese Restaurant',
       'Spa', 'Bike Shop', 'Noodle House', 'New American Restaurant',
       'Salon / Barbershop', 'Bubble Tea Shop', 'Indie Movie Theater',
       'Hotpot Restaurant', 'Tea Room', 'Dessert Shop',
       'American Restaurant', 'Asian Restaurant', 'Roof Deck', 'Museum',
       'Spanish Restaurant', 'Garden Center', 'Historic Site',
       'Thai Restaurant', 'Furniture / Home Store', 'Boutique',
       'Shanghai Restaurant', 'Music Venue', 'Malay Restaurant',
       'Organic Grocery', 'Vietnamese Restaurant', 'Cosmetics Shop',
       'Italian Restaurant', 'Supermarket', 'Optical Shop', 'Snac

<h3>Analyzing Each Neighborhood</h3>

In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

Neighborhood  Accessories Store  Adult Boutique  African Restaurant  \
0  Marble Hill                  0               0                   0   
1  Marble Hill                  0               0                   0   
2  Marble Hill                  0               0                   0   
3  Marble Hill                  0               0                   0   
4  Marble Hill                  0               0                   0   

   American Restaurant  Antique Shop  Arepa Restaurant  \
0                    0             0                 0   
1                    0             0                 0   
2                    0             0                 0   
3                    0             0                 0   
4                    0             0                 0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auditorium  Australian Restaurant  \
0                 0                   0           0                      0   
1                 0                   0           0                      0   
2                 0                   0           0                      0   
3                 0                   0           0                      0   
4                 0                   0           0                      0   

   Austrian Restaurant  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0                    0          0           0           0       0     0    0   
1                    0          0           0           0       0     0    0   
2                    0          0           0           0       0     0    0   
3                    0          0           0           0       0     0    0   
4                    0          0           0           0       0     0    0   

   Baseball Field  Basketball Court  Beer Bar  Beer Garden  Beer Store  \
0               0                 0         0            0           0   
1               0                 0         0            0           0   
2               0                 0         0            0           0   
3               0                 0         0            0           0   
4               0                 0         0            0           0   

   Bike Rental / Bike Share  Bike Shop  Bike Trail  Bistro  Board Shop  \
0                         0          0           0       0           0   
1                         0          0           0       0           0   
2                         0          0           0       0           0   
3                         0          0           0       0           0   
4                         0          0           0       0           0   

   Boat or Ferry  Bookstore  Boutique  Boxing Gym  Brazilian Restaurant  \
0              0          0         0           0                     0   
1              0          0         0           0                     0   
2              0          0         0           0                     0   
3              0          0         0           0                     0   
4              0          0         0           0                     0   

   Breakfast Spot  Bridal Shop  Bridge  Bubble Tea Shop  Building  \
0               0            0       0                0         0   
1               0            0       0                0         0   
2               0            0       0                0         0   
3               0            0       0                0         0   
4               0            0       0                0         0   

   Burger Joint  Burrito Place  Bus Station  Butcher  Caf

In [21]:
manhattan_onehot.shape

(3201, 324)

In [22]:
manhattan_grouped = manhattan_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(manhattan_grouped.shape)
manhattan_grouped

(40, 324)


Neighborhood  Accessories Store  Adult Boutique  \
0     Battery Park City           0.000000            0.00   
1         Carnegie Hill           0.000000            0.00   
2        Central Harlem           0.000000            0.00   
3               Chelsea           0.000000            0.00   
4             Chinatown           0.000000            0.00   
5          Civic Center           0.000000            0.00   
6               Clinton           0.000000            0.00   
7           East Harlem           0.000000            0.00   
8          East Village           0.000000            0.00   
9    Financial District           0.000000            0.00   
10             Flatiron           0.000000            0.00   
11             Gramercy           0.000000            0.00   
12    Greenwich Village           0.010000            0.00   
13     Hamilton Heights           0.000000            0.00   
14         Hudson Yards           0.000000            0.00   
15               Inwood           0.000000            0.00   
16           Lenox Hill           0.000000            0.00   
17       Lincoln Square           0.000000            0.00   
18         Little Italy           0.000000            0.00   
19      Lower East Side           0.000000            0.00   
20     Manhattan Valley           0.000000            0.00   
21       Manhattanville           0.000000            0.00   
22          Marble Hill           0.000000            0.00   
23              Midtown           0.000000            0.00   
24        Midtown South           0.000000            0.00   
25  Morningside Heights           0.000000            0.00   
26          Murray Hill           0.000000            0.00   
27                 Noho           0.000000            0.00   
28     Roosevelt Island           0.000000            0.00   
29                 Soho           0.000000            0.00   
30      Stuyvesant Town           0.000000            0.00   
31         Sutton Place           0.000000            0.01   
32              Tribeca           0.000000            0.00   
33           Tudor City           0.000000            0.00   
34           Turtle Bay           0.000000            0.00   
35      Upper East Side           0.000000            0.00   
36      Upper West Side           0.010204            0.00   
37   Washington Heights           0.011905            0.00   
38         West Village           0.010000            0.00   
39            Yorkville           0.000000            0.00   

    African Restaurant  American Restaurant  Antique Shop  Arepa Restaurant  \
0             0.000000             0.000000          0.00          0.000000   
1             0.000000             0.000000          0.00          0.000000   
2             0.066667             0.044444          0.00          0.000000   
3             0.000000             0.040000          0.00          0.010000   
4             0.000000             0.040000          0.00          0.000000   
5             0.000000             0.020000          0.01          0.000000   
6             0.000000             0.050000          0.00          0.000000   
7             0.000000             0.000000          0.00          0.000000   
8             0.000000             0.020000          0.00          0.000000   
9             0.000000             0.030000          0.00          0.000000   
10            0.000000             0.030000          0.00          0.000000   
11            0.000000             0.042553          0.00          0.000000   
12            0.000000             0.030000          0.00          0.000000   
13            0.000000             0.000000          0.00          0.000000   
14            0.000000             0.048387          0.00          0.000000   
15            0.000000             0.017857          0.00          0.000000   
16            0.000000             0.000000          0.00          0.000000   
17            0.000000             0.021739   

In [23]:
len(manhattan_grouped[manhattan_grouped["Latin American Restaurant"] > 0])

13

In [24]:
manhattan_latin_rest = manhattan_grouped[["Neighborhood","Latin American Restaurant"]]
manhattan_latin_rest

Neighborhood  Latin American Restaurant
0     Battery Park City                   0.000000
1         Carnegie Hill                   0.000000
2        Central Harlem                   0.000000
3               Chelsea                   0.000000
4             Chinatown                   0.000000
5          Civic Center                   0.000000
6               Clinton                   0.000000
7           East Harlem                   0.050000
8          East Village                   0.000000
9    Financial District                   0.000000
10             Flatiron                   0.000000
11             Gramercy                   0.010638
12    Greenwich Village                   0.000000
13     Hamilton Heights                   0.031746
14         Hudson Yards                   0.000000
15               Inwood                   0.017857
16           Lenox Hill                   0.000000
17       Lincoln Square                   0.000000
18         Little Italy                   0.000000
19      Lower East Side                   0.021277
20     Manhattan Valley                   0.020408
21       Manhattanville                   0.000000
22          Marble Hill                   0.000000
23              Midtown                   0.000000
24        Midtown South                   0.000000
25  Morningside Heights                   0.000000
26          Murray Hill                   0.010000
27                 Noho                   0.000000
28     Roosevelt Island                   0.000000
29                 Soho                   0.000000
30      Stuyvesant Town                   0.000000
31         Sutton Place                   0.020000
32              Tribeca                   0.000000
33           Tudor City                   0.012346
34           Turtle Bay                   0.000000
35      Upper East Side                   0.010309
36      Upper West Side                   0.000000
37   Washington Heights                   0.023810
38         West Village                   0.010000
39            Yorkville                   0.010000

<h3>Let's print each neighborhood along with the top 5 most common venues</h3>

In [25]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.09
1          Hotel  0.08
2    Coffee Shop  0.06
3            Gym  0.06
4  Memorial Site  0.05


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.09
1                  Café  0.05
2           Yoga Studio  0.03
3     French Restaurant  0.03
4  Gym / Fitness Center  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Seafood Restaurant  0.04
2  American Restaurant  0.04
3                  Bar  0.04
4   Chinese Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1  American Restaurant  0.04
2          Art Gallery  0.04
3               Bakery  0.04
4   Italian Restaurant  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.08
1               Bakery  0.06
2         Cocktail Bar  0.05
3  American Restaurant  0.04
4   Salon / Barbershop  0.03


----Civic Center----
                  venue 

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  Battery Park City                  Park                 Hotel   
1      Carnegie Hill           Coffee Shop                  Café   
2     Central Harlem    African Restaurant    Seafood Restaurant   
3            Chelsea           Coffee Shop                Bakery   
4          Chinatown    Chinese Restaurant                Bakery   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Gym           Coffee Shop         Memorial Site   
1             Bookstore    Italian Restaurant  Gym / Fitness Center   
2    Chinese Restaurant   American Restaurant        Cosmetics Shop   
3   American Restaurant           Art Gallery                  Café   
4          Cocktail Bar   American Restaurant          Noodle House   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Clothing Store          Burger Joint                 Plaza   
1                   Gym     French Restaurant           Yoga Studio   
2                   Bar     French Restaurant                   Gym   
3    Italian Restaurant                  Park          Cycle Studio   
4   Shanghai Restaurant    Salon / Barbershop          Dessert Shop   

  9th Most Common Venue 10th Most Common Venue  
0          Gourmet Shop             Playground  
1             Wine Shop  Vietnamese Restaurant  
2             Bookstore               Boutique  
3          Cupcake Shop                 Market  
4          Optical Shop                    Spa

<h3>Clustering Neighborhoods</h3>

In [29]:
kclusters = 5

manhattan_clustering = manhattan_latin_rest.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 3, 2, 2])

In [30]:
manhattan_merged = manhattan_latin_rest.copy()

# add clustering labels
manhattan_merged["Cluster Labels"] = kmeans.labels_

In [31]:
manhattan_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
manhattan_merged.head()

Neighborhood  Latin American Restaurant  Cluster Labels
0  Battery Park City                        0.0               2
1      Carnegie Hill                        0.0               2
2     Central Harlem                        0.0               2
3            Chelsea                        0.0               2
4          Chinatown                        0.0               2

In [32]:
manhattan_merged = manhattan_merged.join(manhattan_data.set_index("Neighborhood"), on="Neighborhood")

print(manhattan_merged.shape)
manhattan_merged.head() # check the last columns!

(40, 6)


Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
0  Battery Park City                        0.0               2  Manhattan   
1      Carnegie Hill                        0.0               2  Manhattan   
2     Central Harlem                        0.0               2  Manhattan   
3            Chelsea                        0.0               2  Manhattan   
4          Chinatown                        0.0               2  Manhattan   

    Latitude  Longitude  
0  40.711932 -74.016869  
1  40.782683 -73.953256  
2  40.815976 -73.943211  
3  40.744035 -74.003116  
4  40.715618 -73.994279

In [33]:
print(manhattan_merged.shape)
manhattan_merged.sort_values(["Cluster Labels"], inplace=True)
manhattan_merged

(40, 6)


Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
39            Yorkville                   0.010000               0  Manhattan   
35      Upper East Side                   0.010309               0  Manhattan   
33           Tudor City                   0.012346               0  Manhattan   
26          Murray Hill                   0.010000               0  Manhattan   
38         West Village                   0.010000               0  Manhattan   
11             Gramercy                   0.010638               0  Manhattan   
37   Washington Heights                   0.023810               1  Manhattan   
31         Sutton Place                   0.020000               1  Manhattan   
20     Manhattan Valley                   0.020408               1  Manhattan   
15               Inwood                   0.017857               1  Manhattan   
19      Lower East Side                   0.021277               1  Manhattan   
9    Financial District                   0.000000               2  Manhattan   
1         Carnegie Hill                   0.000000               2  Manhattan   
36      Upper West Side                   0.000000               2  Manhattan   
2        Central Harlem                   0.000000               2  Manhattan   
34           Turtle Bay                   0.000000               2  Manhattan   
3               Chelsea                   0.000000               2  Manhattan   
32              Tribeca                   0.000000               2  Manhattan   
4             Chinatown                   0.000000               2  Manhattan   
30      Stuyvesant Town                   0.000000               2  Manhattan   
29                 Soho                   0.000000               2  Manhattan   
28     Roosevelt Island                   0.000000               2  Manhattan   
27                 Noho                   0.000000               2  Manhattan   
5          Civic Center                   0.000000               2  Manhattan   
10             Flatiron                   0.000000               2  Manhattan   
25  Morningside Heights                   0.000000               2  Manhattan   
23              Midtown                   0.000000               2  Manhattan   
22          Marble Hill                   0.000000               2  Manhattan   
21       Manhattanville                   0.000000               2  Manhattan   
6               Clinton                   0.000000               2  Manhattan   
18         Little Italy                   0.000000               2  Manhattan   
17       Lincoln Square                   0.000000               2  Manhattan   
16           Lenox Hill                   0.000000               2  Manhattan   
8          East Village                   0.000000               2  Manhattan   
14         Hudson Yards                   0.000000               2  Manhattan   
12    Greenwich Village                   0.000000               2  Manhattan   
24        Midtown South                   0.000000               2  Manhattan   
0     Battery Park City                   0.000000               2  Manhattan   
7           East Harlem                   0.050000               3  Manhattan   
13     Hamilton Heights                   0.031746               4  Manhattan   

     Latitude  Longitude  
39  40.775930 -73.947118  
35  40.775639 -73.960508  
33  40.746917 -73.971219  
26  40.748303 -73.978332  
38  40.734434 -74.006180  
11  40.737210 -73.981376  
37  40.851903 -73.936900  
31  40.760280 -73.963556  
20  40.797307 -73.964286  
15  40.867684 -73.921210  
19  40.717807 -73.980890  
9   40.707107 -74.010665  
1   40.782683 -73.953256  
36  40.787658 -73.977059  
2   40.815976 -73.943211  
34  40.752042 -73.967708  
3   40.744035 -74.003116  
32  40.721522 -74.010683  
4   40.715618 -73.994279  
30  40.731000 -73.974052  
29  40.722184 -74.000657  
28  40.762160 -73.949168  
27  40.723259 -73.988434  
5   40.715229 -74.005415  
10  40.739673 

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examining Clusters</h3>

In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]

Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
39        Yorkville                   0.010000               0  Manhattan   
35  Upper East Side                   0.010309               0  Manhattan   
33       Tudor City                   0.012346               0  Manhattan   
26      Murray Hill                   0.010000               0  Manhattan   
38     West Village                   0.010000               0  Manhattan   
11         Gramercy                   0.010638               0  Manhattan   

     Latitude  Longitude  
39  40.775930 -73.947118  
35  40.775639 -73.960508  
33  40.746917 -73.971219  
26  40.748303 -73.978332  
38  40.734434 -74.006180  
11  40.737210 -73.981376

In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1]

Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
37  Washington Heights                   0.023810               1  Manhattan   
31        Sutton Place                   0.020000               1  Manhattan   
20    Manhattan Valley                   0.020408               1  Manhattan   
15              Inwood                   0.017857               1  Manhattan   
19     Lower East Side                   0.021277               1  Manhattan   

     Latitude  Longitude  
37  40.851903 -73.936900  
31  40.760280 -73.963556  
20  40.797307 -73.964286  
15  40.867684 -73.921210  
19  40.717807 -73.980890

In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2]

Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
9    Financial District                        0.0               2  Manhattan   
1         Carnegie Hill                        0.0               2  Manhattan   
36      Upper West Side                        0.0               2  Manhattan   
2        Central Harlem                        0.0               2  Manhattan   
34           Turtle Bay                        0.0               2  Manhattan   
3               Chelsea                        0.0               2  Manhattan   
32              Tribeca                        0.0               2  Manhattan   
4             Chinatown                        0.0               2  Manhattan   
30      Stuyvesant Town                        0.0               2  Manhattan   
29                 Soho                        0.0               2  Manhattan   
28     Roosevelt Island                        0.0               2  Manhattan   
27                 Noho                        0.0               2  Manhattan   
5          Civic Center                        0.0               2  Manhattan   
10             Flatiron                        0.0               2  Manhattan   
25  Morningside Heights                        0.0               2  Manhattan   
23              Midtown                        0.0               2  Manhattan   
22          Marble Hill                        0.0               2  Manhattan   
21       Manhattanville                        0.0               2  Manhattan   
6               Clinton                        0.0               2  Manhattan   
18         Little Italy                        0.0               2  Manhattan   
17       Lincoln Square                        0.0               2  Manhattan   
16           Lenox Hill                        0.0               2  Manhattan   
8          East Village                        0.0               2  Manhattan   
14         Hudson Yards                        0.0               2  Manhattan   
12    Greenwich Village                        0.0               2  Manhattan   
24        Midtown South                        0.0               2  Manhattan   
0     Battery Park City                        0.0               2  Manhattan   

     Latitude  Longitude  
9   40.707107 -74.010665  
1   40.782683 -73.953256  
36  40.787658 -73.977059  
2   40.815976 -73.943211  
34  40.752042 -73.967708  
3   40.744035 -74.003116  
32  40.721522 -74.010683  
4   40.715618 -73.994279  
30  40.731000 -73.974052  
29  40.722184 -74.000657  
28  40.762160 -73.949168  
27  40.723259 -73.988434  
5   40.715229 -74.005415  
10  40.739673 -73.990947  
25  40.808000 -73.963896  
23  40.754691 -73.981669  
22  40.876551 -73.910660  
21  40.816934 -73.957385  
6   40.759101 -73.996119  
18  40.719324 -73.997305  
17  40.773529 -73.985338  
16  40.768113 -73.958860  
8   40.727847 -73.982226  
14  40.756658 -74.000111  
12  40.726933 -73.999914  
24  40.748510 -73.988713  
0   40.711932 -74.016869

In [38]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3]

Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
7  East Harlem                       0.05               3  Manhattan   

    Latitude  Longitude  
7  40.792249 -73.944182

In [39]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4]

Neighborhood  Latin American Restaurant  Cluster Labels    Borough  \
13  Hamilton Heights                   0.031746               4  Manhattan   

     Latitude  Longitude  
13  40.823604 -73.949688